### Calculating the sensitivity of SSAM output to small fluctuations in input
Quantifying differences in the output files of Salish Sea Atlantis Model simulations with small differences in input

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import ssam_groups as groups
import ssam_plot as splot

In [32]:
def my_rmsd_percent(one, two):
    one_all_layers = one.sum(axis=2)
    two_all_layers = two.sum(axis=2)
    diff = abs(one_all_layers-two_all_layers)
    diff_ratio = diff/two_all_layers
    diff_ratio_squared = diff_ratio**2
    diff_ratio_squared1 = diff_ratio_squared.mean(axis=1, skipna=True)
    diff_ratio_squared_mean = diff_ratio_squared1.mean(axis=0, skipna=True)
    my_rmsd_percent = ((diff_ratio_squared_mean)**(1/2))*100
    return my_rmsd_percent

In [58]:
def my_rmsd_percent_resampled(one, two):
    one_column = one.sum(dim='z')
    two_column = two.sum(dim='z')
    one_sea = one_column.sum(dim='b')
    two_sea = two_column.sum(dim='b')
    diff = abs(one_sea-two_sea)
    diff_ratio = diff/two_sea
    diff_ratio_squared = diff_ratio**2
    diff_ratio_squared_mean = diff_ratio_squared.mean(dim='t', skipna=True)
    my_rmsd_percent_resampled = ((diff_ratio_squared_mean)**(1/2))*100
    return my_rmsd_percent_resampled

In [44]:
#control_file = "/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Sensitivity/control/outputSalishSea.nc"
control_file = "/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Scrubber/Scrubber_discharge_100y_2029_latestart/outputSalishSea.nc"
control= xr.open_dataset(str(control_file), decode_cf=True)
annual_control = control.resample(t='5Y').mean()

In [36]:
scenario_path = Path('/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Scrubber/')

In [35]:
# Masking land boxes
Temps = np.ma.filled(control.Temp[1,:,5], np.nonzero)
(ocean_boxes) = Temps.nonzero()
ocean_boxes = ocean_boxes[0]
ocean_boxes = ocean_boxes[1:ocean_boxes.size-1]
#ocean_boxes

In [39]:
scens = sorted([p for p in scenario_path.glob('Scrubber_discharge_100y_2024*/outputSalishSea.nc')])
for path in scens:
    nm = str(path).split(sep = '/')
    #name1 = nm[7].split(sep='_')
    name = nm[7]
name

'Scrubber_discharge_100y_2024_latestart'

Process files with snapshot data

In [45]:
#file_dict = {'BC', 'FHE', 'MA', 'ORR', 'PL', 'PS', 'WHB'}
file_dict = {'MA', 'PS'}
scens = sorted([p for p in scenario_path.glob('Scrubber_discharge_100y_2029/outputSalishSea.nc')])

# measured opening of scenario files
for path in scens:
    nm = str(path).split(sep = '/')
    name = nm[7]
    #name1 = nm[7].split(sep='_')
    #name = name1[1]

    rmsd_data = pd.DataFrame({'scenario_file': [],
                    'species': [],
                    'RMSD%': []},)
    scenario = xr.open_dataset(str(path), decode_cf=True)
    for species in groups.all_pelagic:
        one = control.variables[groups.all_pelagic[species] + '_N'][:][ocean_boxes]
        two = scenario.variables[groups.all_pelagic[species] +'_N'][:][ocean_boxes]

        rmsd_score = my_rmsd_percent(one, two)
        rmsd_data.loc[len(rmsd_data.index)] = [name, species, float(rmsd_score)]                       
    rmsd_data.to_csv("/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Sensitivity/rmsd_percent_snapshot"+name+"_13Jul.csv")

Process files with 5-year resampled data

In [59]:
#file_dict = {'BC', 'FHE', 'MA', 'ORR', 'PL', 'PS', 'WHB'}
#file_dict = {'MA', 'PS'}

#scens = sorted([p for p in scenario_path.glob('butterflyeffect_*x11/outputSalishSea.nc')])

# measured opening of scenario files
for path in scens:
    nm = str(path).split(sep = '/')
    name = nm[7]
    #name1 = nm[7].split(sep='_')
    #name = name1[1]

    rmsd_data = pd.DataFrame({'scenario_file': [],
                    'species': [],
                    'RMSD%': []},)
    scenario = xr.open_dataset(str(path), decode_cf=True)
    annual_scenario = scenario.resample(t='5Y').mean()
    for species in groups.all_pelagic:
        two = annual_scenario.variables[groups.all_pelagic[species] +'_N'][:, ocean_boxes, :]
        one = annual_control.variables[groups.all_pelagic[species] + '_N'][:, ocean_boxes, :]

        rmsd_score = my_rmsd_percent_resampled(one, two)
        rmsd_data.loc[len(rmsd_data.index)] = [name, species, float(rmsd_score)]                       
    rmsd_data.to_csv("/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Sensitivity/rmsd_percent_resampled2_"+name+"_13Jul.csv")

Aggregating planktonic biomass

In [47]:
def my_rmsd_percent_sum(one, two):
    one = one.sum(dim='z')
    two = two.sum(dim='z')
    one = one.sum(dim='b')
    two = two.sum(dim='b')
    one = one.sum(dim='t')
    two = two.sum(dim='t')
    diff_ratio = (abs(one-two)/two)
    diff_ratio_squared = diff_ratio**2
    my_rmsd_percent_sum = ((diff_ratio_squared)**(1/2))*100
    return my_rmsd_percent_sum

In [51]:
for path in scens:
    nm = str(path).split(sep = '/')
    name = nm[7]
    #name1 = nm[7].split(sep='_')
    #name = name1[1]

    rmsd_data = pd.DataFrame({'scenario_file': [],
                    'species': [],
                    'RMSD%': []},)
    scenario = xr.open_dataset(str(path), decode_cf=True)
    annual_scenario = scenario.resample(t='5Y').mean()
    for species in groups.plankton:
        two = annual_scenario.variables[groups.plankton[species] +'_N'][:, ocean_boxes, :]
        one = annual_control.variables[groups.plankton[species] + '_N'][:, ocean_boxes, :]

        rmsd_score = my_rmsd_percent_sum(one, two)
        rmsd_data.loc[len(rmsd_data.index)] = [name, species, float(rmsd_score)]                       
    rmsd_data.to_csv("/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Sensitivity/rmsd_percent_sum_"+name+"_13Jul.csv")

In [52]:
def my_rmsd_percent_sum2(one, two):
    one = one.sum(axis=2)
    two = two.sum(axis=2)
    one = one.sum(axis=1)
    two = two.sum(axis=1)
    one = one.sum(axis=0)
    two = two.sum(axis=0)
    diff_ratio = (abs(one-two)/two)
    diff_ratio_squared = diff_ratio**2
    my_rmsd_percent_sum2 = ((diff_ratio_squared)**(1/2))*100
    return my_rmsd_percent_sum2

In [53]:
for path in scens:
    nm = str(path).split(sep = '/')
    name = nm[7]
    #name1 = nm[7].split(sep='_')
    #name = name1[1]

    rmsd_data = pd.DataFrame({'scenario_file': [],
                    'species': [],
                    'RMSD%': []},)
    scenario = xr.open_dataset(str(path), decode_cf=True)
    for species in groups.all_pelagic:
        one = control.variables[groups.all_pelagic[species] + '_N'][:][ocean_boxes]
        two = scenario.variables[groups.all_pelagic[species] +'_N'][:][ocean_boxes]

        rmsd_score = my_rmsd_percent_sum2(one, two)
        rmsd_data.loc[len(rmsd_data.index)] = [name, species, float(rmsd_score)]                       
    rmsd_data.to_csv("/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Sensitivity/rmsd_percent_sum2_"+name+"_13Jul.csv")

In [54]:
def benthic_rmsd_percent_resampled(one, two):
    one_sea = one.sum(dim='b')
    two_sea = two.sum(dim='b')
    diff = abs(one_sea-two_sea)
    diff_ratio = diff/two_sea
    diff_ratio_squared = diff_ratio**2
    diff_ratio_squared_mean = diff_ratio_squared.mean(dim='t', skipna=True)
    benthic_rmsd_percent_resampled = ((diff_ratio_squared_mean)**(1/2))*100
    return benthic_rmsd_percent_resampled

In [57]:
for path in scens:
    nm = str(path).split(sep = '/')
    name = nm[7]
    #name1 = nm[7].split(sep='_')
    #name = name1[1]

    rmsd_data = pd.DataFrame({'scenario_file': [],
                    'species': [],
                    'RMSD%': []},)
    scenario = xr.open_dataset(str(path), decode_cf=True)
    annual_scenario = scenario.resample(t='5Y').mean()
    for species in groups.benthos:
        two = annual_scenario.variables[groups.benthos[species] +'_N'][:, ocean_boxes]
        one = annual_control.variables[groups.benthos[species] + '_N'][:, ocean_boxes]

        rmsd_score = benthic_rmsd_percent_resampled(one, two)
        rmsd_data.loc[len(rmsd_data.index)] = [name, species, float(rmsd_score)]                       
    rmsd_data.to_csv("/ocean/rlovindeer/MOAD/analysis-raisha/SSmodel_outputs/Sensitivity/rmsd_benthos_resampled_"+name+"_13Jul.csv")